### Name   : Ibadullah Hayat
### Reg No : B23F0001AI010
### Section : F23 AI-GREEN

# Lab 02: Data Wrangling & Visualization

Objective: Practice real-world data cleaning, filtering, and visualization using Pandas and Matplotlib on the IMDb Top 1000 Movies dataset.

### Task 1: The Data Cleaner

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the IMDb Top 1000 movies dataset
movie_df = pd.read_csv('imdb_top_1000.csv')
print("Dataset loaded successfully!")
print(f"Dataset shape: {movie_df.shape}")

I start by loading the dataset. It contains 1000 top-rated movies with details like title, director, rating, votes, revenue, etc.

In [ ]:
# 1.1: Count missing values in 'Meta_score'
missing_metascore = movie_df['Meta_score'].isnull().sum()
print(f"\n {missing_metascore} movies are missing a Metascore value.")

Many movies don’t have a Metascore (critic review aggregate). This is common in real-world data—some fields are optional or unavailable.

In [ ]:
# 1.2: Clean and fill missing 'Gross' values with median
df_clean = movie_df.copy()

# Remove commas from 'Gross' and convert to numeric
df_clean['Gross'] = (
    df_clean['Gross']
    .astype(str)
    .str.replace(',', '', regex=False)
    .replace('nan', np.nan)
)
df_clean['Gross'] = pd.to_numeric(df_clean['Gross'], errors='coerce')

# Fill NaN with median revenue
median_gross = df_clean['Gross'].median()
df_clean['Gross'].fillna(median_gross, inplace=True)
print(f" Missing Gross values filled with median: ${median_gross:,.2f}")

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


The Gross column has commas (e.g., "23,530,892"), so I remove them before converting to numbers. Then I fill missing values with the median (not mean!) because revenue is skewed—median is more robust to outliers.

In [1]:
# 1.3: Filter movies from 2012 with IMDB Rating > 7.5
# Ensure 'Released_Year' is numeric
df_clean['Released_Year'] = pd.to_numeric(df_clean['Released_Year'], errors='coerce')

# Apply filter
filtered_2012 = df_clean[
    (df_clean['Released_Year'] == 2012) &
    (df_clean['IMDB_Rating'] > 7.5)
]

print(f"\n Found {len(filtered_2012)} movies from 2012 with rating > 7.5:")
print(filtered_2012[['Series_Title', 'Released_Year', 'IMDB_Rating', 'Director']].to_string(index=False))

NameError: name 'pd' is not defined

I convert Released_Year to numeric to enable comparison. Then I use boolean indexing to filter rows that meet both conditions. This shows how powerful Pandas is for querying data.

Reflection: Real datasets are messy. Handling missing values and type conversions is essential before any analysis. Using .fillna() with a sensible strategy (like median) prevents bias in downstream tasks.



### Task 2: The Visualizer

2.1: Scatter Plot – Rating vs. Revenue

In [ ]:
# Scatter plot: Rating vs Gross Revenue
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['IMDB_Rating'], df_clean['Gross'], alpha=0.6, color='steelblue', s=40)

# Calculate correlation
corr = df_clean['IMDB_Rating'].corr(df_clean['Gross'])
plt.figtext(0.15, 0.85, f'Correlation: {corr:.3f}',
            fontsize=12, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))

plt.title('Movie Rating vs Gross Revenue', fontsize=14, fontweight='bold')
plt.xlabel('IMDB Rating')
plt.ylabel('Gross Revenue (USD)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('rating_vs_revenue.png', dpi=150)
plt.show()

print(f"\n Correlation between rating and revenue: {corr:.3f}")
print(" Insight: Weak positive correlation — higher-rated movies tend to earn more, but not strongly.")

   157 movies are missing a Metascore value.



The scatter plot shows a weak positive correlation (r ≈ 0.086). While critically acclaimed films often do well, box office success depends heavily on marketing, genre, and star power—not just quality.

2.2: Bar Chart – Top 5 Directors

In [ ]:
# Top 5 directors by movie count
top_directors = df_clean['Director'].value_counts().head(5)

plt.figure(figsize=(10, 6))
bars = plt.bar(top_directors.index, top_directors.values, color='teal')
plt.title('Top 5 Directors with Most Movies in IMDb Top 1000', fontsize=14, fontweight='bold')
plt.xlabel('Director')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             str(int(bar.get_height())), ha='center', va='bottom', fontweight='bold')

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('top_directors.png', dpi=150)
plt.show()

print("\n🏆 Top 5 Directors:")
for director, count in top_directors.items():
    print(f" - {director}: {count} movies")

Legendary directors like Alfred Hitchcock (14) and Steven Spielberg (13) dominate the list. This reflects their consistent high-quality output over decades.

2.3: Histogram – Distribution of Votes (Log Scale)

In [ ]:
# Histogram of votes (log scale for clarity)
plt.figure(figsize=(10, 6))
plt.hist(df_clean['No_of_Votes'], bins=30, color='royalblue', edgecolor='black', alpha=0.7)
plt.yscale('log')  # Log scale for heavy-tailed distribution

mean_votes = df_clean['No_of_Votes'].mean()
median_votes = df_clean['No_of_Votes'].median()

plt.axvline(mean_votes, color='red', linestyle='--', label=f'Mean: {mean_votes:,.0f}')
plt.axvline(median_votes, color='green', linestyle='--', label=f'Median: {median_votes:,.0f}')

plt.title('Distribution of Movie Votes (Log Scale)', fontsize=14, fontweight='bold')
plt.xlabel('Number of Votes')
plt.ylabel('Frequency (Log Scale)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('votes_distribution.png', dpi=150)
plt.show()

print(f"\n Mean votes: {mean_votes:,.0f}")
print(f" Median votes: {median_votes:,.0f}")
print(" Insight: Heavy-tailed distribution — most movies have few votes, a few have millions.")

The log-scale histogram reveals a heavy-tailed distribution: most films have modest engagement, while blockbusters like The Dark Knight attract millions of votes. The mean > median confirms right-skewness.

### Conclusion
This lab taught me how to:

Clean real-world data (handle missing values, type conversion)
Filter with boolean logic (multi-condition queries)
Visualize relationships and distributions effectively
Key insights:

Critical acclaim (rating) only weakly predicts box office success.
A handful of legendary directors dominate the IMDb Top 1000.
Vote counts follow a power-law distribution—common in popularity metrics.
These skills are foundational for any machine learning pipeline, where data quality determines model performance.